# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 생성
prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘"
)
prompt = prompt_template.format_prompt(
    topic="호랑이",
    count=2
)

# LLM 모델 생성
model = ChatOpenAI(model="gpt-4o-mini")

# LLM 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt)

In [4]:
response.content

'한국의 호랑이에 관련된 속담 두 가지는 다음과 같습니다:\n\n1. **"호랑이 굴에 가야 호랑이 새끼를 잡는다."**  \n   이 속담은 어떤 목표를 이루기 위해서는 위험을 감수하고 적극적으로 행동해야 한다는 의미입니다. 즉, 좋은 결과를 얻기 위해서는 도전이 필요하다는 것을 강조합니다.\n\n2. **"호랑이보다 더 무서운 것은 사람이다."**  \n   이 속담은 사람의 힘이나 악의가 호랑이 같은 짐승보다 더 위협적일 수 있다는 뜻입니다. 즉, 외부의 위협보다도 인간의 행동이 더 무서울 수 있다는 경각심을 담고 있습니다.\n\n이 두 가지 속담은 호랑이를 상징적으로 사용하여 인간의 행동과 선택에 대한 교훈을 전달합니다.'

In [5]:
parser = StrOutputParser()                                  # Message 객체에서 content 속성의 값만 추출
res = parser.parse(response)                                # 응답을 파싱하여 문자열로 변환
print(res)                                                  # 결과 출력

content='한국의 호랑이에 관련된 속담 두 가지는 다음과 같습니다:\n\n1. **"호랑이 굴에 가야 호랑이 새끼를 잡는다."**  \n   이 속담은 어떤 목표를 이루기 위해서는 위험을 감수하고 적극적으로 행동해야 한다는 의미입니다. 즉, 좋은 결과를 얻기 위해서는 도전이 필요하다는 것을 강조합니다.\n\n2. **"호랑이보다 더 무서운 것은 사람이다."**  \n   이 속담은 사람의 힘이나 악의가 호랑이 같은 짐승보다 더 위협적일 수 있다는 뜻입니다. 즉, 외부의 위협보다도 인간의 행동이 더 무서울 수 있다는 경각심을 담고 있습니다.\n\n이 두 가지 속담은 호랑이를 상징적으로 사용하여 인간의 행동과 선택에 대한 교훈을 전달합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 22, 'total_tokens': 207, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgNFnQQRsh11QqHOlSp3Sz7IX1cqm', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a069e303-db52-4129-8f6c-928249fc1aa7-0' usage_metadata={'input_tokens': 22, 'output_

In [6]:
# prompt_template -> model -> response -> output parser
chain = prompt_template | model | parser                    # 체인 생성(pipeline)

res = chain.invoke({"topic": "사람의 정신력", "count": 3})
print(res)

한국에는 정신력과 관련된 여러 속담이 있습니다. 다음은 그 중 세 가지입니다.

1. **"떡 줄 사람은 꿈도 안 꾸고"**  
   의미: 무언가를 받으려면 그에 상응하는 준비나 노력이 필요하다는 것을 나타냅니다. 힘든 상황에서도 포기하지 않으려는 정신력과 노력의 중요성을 강조합니다.

2. **"고생 끝에 낙이 온다"**  
   의미: 힘든 과정을 겪고 나면 좋은 결과가 뒤따른다는 뜻으로, 힘든 순간에도 최선을 다하면 결국에는 보상이 있다는 것을 알려줍니다.

3. **"웃으면 복이 온다"**  
   의미: 긍정적인 마음가짐이 좋은 운을 가져온다는 뜻으로, 어려운 상황에서도 긍정적인 태도를 유지하는 것이 중요하다는 것을 강조합니다.

이 속담들은 어려운 상황에서의 정신력과 긍정적인 태도의 중요성을 잘 보여줍니다.


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [7]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()
res_txt = "사과,당근,수박,참외,메론"
print(parser.invoke(res_txt))

['사과', '당근', '수박', '참외', '메론']


In [8]:
# 출력 형식 지정
format_string = parser.get_format_instructions()
print(format_string)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [9]:
from textwrap import dedent
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
        # instruction
        {subject}의 이름 다섯개를 나열해주세요.

        # output indicator  ->  출력 형식 지정
        {format_instructions}
        """),
        partial_variables={"format_instructions":parser.get_format_instructions()}
)
# partial_variables={변수명:넣을값,...} : 템플릿의 placeholder 변수에 넣을 값을 prompttemplate 객체 생성 시 넣음
# placeholder에 넣을 값이 있는데 함수나 메소드호출을 통해 그 값을 가져와야 하는 경우 사용

model = ChatOpenAI(model_name="gpt-4o-mini")

# prompt 생성
prompt = prompt_template.invoke({"subject":"동물"})
# LLM에 요청
response = model.invoke(prompt)

In [10]:
print(response.content)
res = parser.invoke(response)
print(type(res))
print(res)

고양이, 개, 토끼, 코끼리, 사자
<class 'list'>
['고양이', '개', '토끼', '코끼리', '사자']


In [11]:
# chain으로 구성
chain = prompt_template | model | parser

res = chain.invoke({"subject":"한국산 자동차"})
res

['소나타', '아반떼', '말리부', '싼타페', '투싼']

## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [12]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
res_text = """
    {
        "name":"홍길동",
        "age":20,
        "address":"서울",
        "hobby":["독서", "게임"]
    }
"""
res_dict = parser.invoke(res_text)
print(type(res_dict))
res_dict

<class 'dict'>


{'name': '홍길동', 'age': 20, 'address': '서울', 'hobby': ['독서', '게임']}

In [13]:
parser.get_format_instructions()

'Return a JSON object.'

In [14]:
prompt_template = ChatPromptTemplate.from_template(
    "{name}에 대해서 설명해줘.\n{format_instructions}",
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
model = ChatOpenAI(model_name="gpt-4o-mini")
prompt = prompt_template.invoke({"name":"아이폰"})
response = model.invoke(prompt)

In [15]:
print(response.content)

```json
{
  "product": {
    "name": "아이폰",
    "manufacturer": "Apple Inc.",
    "first_release_date": "2007-06-29",
    "current_model": {
      "name": "아이폰 15",
      "release_date": "2023-09-22",
      "features": [
        "A17 Pro 칩",
        "Super Retina XDR 디스플레이",
        "5G 연결",
        "향상된 카메라 시스템",
        "다양한 색상 옵션"
      ]
    },
    "operating_system": {
      "name": "iOS",
      "current_version": "iOS 17",
      "features": [
        "개인정보 보호 기능 향상",
        "위젯 및 홈 화면 맞춤 설정",
        "FaceTime 및 메시지의 새로운 기능"
      ]
    },
    "significance": "아이폰은 스마트폰의 본질을 변화시킨 혁신적인 제품으로, 터치스크린 기반의 인터페이스와 앱 생태계를 통해 모바일 기술을 발전시키는 데 기여했습니다.",
    "market_impact": "아이폰은 전 세계 스마트폰 시장에서 중요한 위치를 차지하고 있으며, Apple의 주요 수익원 중 하나입니다."
  }
}
```


In [16]:
response_dict = parser.invoke(response)
print(type(response.content), type(response_dict))

<class 'str'> <class 'dict'>


In [17]:
# response_dict["아이폰"]["첫 번째 출시"]

In [20]:
# 출력 스키마를 정의
# json 형식을 설계
from pydantic import BaseModel, Field

class ItemSchema(BaseModel):
    # JSON에 포함될 항목들을 class변수로 정의. 변수명:타입=Field(설명)
    name: str = Field(description="제품의 이름")
    info: str = Field(description="제품에 대한 정보")
    release_date: str = Field(description="제품이 출시된 일시, yyyy-mm-dd 형식")
    price: int = Field(description="제품의 가격")

parser = JsonOutputParser(pydantic_object=ItemSchema)
# print(parser.get_format_instructions())

prompt_template = ChatPromptTemplate.from_template(
    dedent("""
        # instruction
        {name}에 대해서 설명해주세요.
           
        # output indicator
        {format_instructions}
"""),
partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

In [24]:
p = prompt_template.invoke({"name":"갤럭시s24"})
res = model.invoke(p)
response = parser.invoke(res)
response

{'name': '갤럭시 S24',
 'info': '갤럭시 S24는 삼성의 최신 스마트폰으로, 강력한 프로세서와 향상된 카메라 성능을 제공합니다. 또한, 120Hz 주사율의 AMOLED 디스플레이와 IP68등급의 방수 및 방진 기능이 특징입니다.',
 'release_date': '2023-02-01',
 'price': 999}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [25]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ItemSchema)
# JsonOutputParser와 동일한 format instruction을 생성
# 응답 : JSON -> Parser : Pydantic Model 객체로 변환
print(parser.get_format_instructions)

<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.ItemSchema'>)>


In [26]:
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
        # instruction
        {name}에 대해서 설명해주세요.
           
        # output indicator
        {format_instructions}
"""),
partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

In [40]:
prompt = prompt_template.invoke({"name":"맥북"})
res = model.invoke(prompt)
response = parser.invoke(res)

In [41]:
type(response)

__main__.ItemSchema

In [42]:
print("제품이름", response.name)
print("정보", response.info)
print(response.release_date, response.price)

제품이름 맥북
정보 애플사가 개발한 노트북 컴퓨터로, macOS 운영체제를 실행하며 우수한 디자인과 성능을 자랑합니다.
2006-01-10 1200000


In [43]:
chain = prompt_template | model | parser
response = chain.invoke({"name":"아이폰"})

In [44]:
response

ItemSchema(name='아이폰', info='아이폰은 애플이 개발한 스마트폰으로, 터치스크린 인터페이스와 iOS 운영 체제를 채택하고 있으며, 다양한 애플리케이션을 지원합니다.', release_date='2007-06-29', price=999)